In [1]:
!wget https://github.com/fernandoandreotti/cinc-challenge2017/raw/master/deeplearn-approach/ResNet_30s_34lay_16conv.hdf5

--2019-05-12 21:44:33--  https://github.com/fernandoandreotti/cinc-challenge2017/raw/master/deeplearn-approach/ResNet_30s_34lay_16conv.hdf5
Resolving github.com... 13.229.188.59
Connecting to github.com|13.229.188.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fernandoandreotti/cinc-challenge2017/master/deeplearn-approach/ResNet_30s_34lay_16conv.hdf5 [following]
--2019-05-12 21:44:34--  https://raw.githubusercontent.com/fernandoandreotti/cinc-challenge2017/master/deeplearn-approach/ResNet_30s_34lay_16conv.hdf5
Resolving raw.githubusercontent.com... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10604536 (10M) [application/octet-stream]
Saving to: ‘ResNet_30s_34lay_16conv.hdf5.1’

ResNet_30s_34lay_16 100%[===================>]  10.11M   826KB/s    in 14s     

2019-05-12 21:44:50 (749 KB/

In [2]:
from random import randint
import urllib.request
import scipy.io
from keras.models import load_model
import numpy as np
import pandas as pd

Using TensorFlow backend.


### Load Model from hdfd

The model was trained and I only reused back from here https://github.com/fernandoandreotti/cinc-challenge2017/tree/master/deeplearn-approach

#### The classes
* A = AF
* N = Normal
* O = Others
* ~ = Noisy

In [3]:
%%time
# load from trained model
print("Loading model")    
model = load_model('ResNet_30s_34lay_16conv.hdf5')
classes = ['A', 'N', 'O','~']

Loading model
CPU times: user 21.4 s, sys: 365 ms, total: 21.8 s
Wall time: 23.7 s


//anaconda/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [4]:
# randomly choose data from the physiobank
record = "A{:05d}".format(randint(0, 999))
urlfile = "https://www.physionet.org/physiobank/database/challenge/2017/training/A00/{}.mat".format(record)
local_filename, headers = urllib.request.urlretrieve(urlfile)
html = open(local_filename)
print('Downloading record {} ..'.format(record))

In [5]:
def preprocess(data):
    # Preprocessing data
    maxlen = 9000 # the model expect (9000, 1) shape
    print("Preprocessing recording ..")    
    X = np.zeros((1,maxlen))
    data = np.nan_to_num(data) # removing NaNs and Infs
    data = data[0:maxlen]
    data = data - np.mean(data)
    data = data/np.std(data)
    X[0,:len(data)] = data.T # padding sequence
    data = X
    data = np.expand_dims(data, axis=2) # required by Keras
    del X
    return data

In [6]:
mat_data = scipy.io.loadmat(local_filename)
data = mat_data['val']
data = data.reshape(data.shape[1])

In [7]:
data.shape

(9000,)

In [8]:
print("Applying model ..")   
data = preprocess(data)
prob = model.predict(data)
ann = np.argmax(prob)
print("Record {} classified as {} with {:3.1f}% certainty".format(record,classes[ann],100*prob[0,ann]))

Applying model ..
Preprocessing recording ..
Record A00906 classified as O with 66.5% certainty


### Let's try on our saved filtered ECG data

In [9]:
df = pd.read_csv("ecg_filtered.csv")

In [10]:
df.head()

,ECG_Filtered
0,18.326446
1,30.774347
2,42.439289
3,50.279598
4,52.229285


In [11]:
data = df['ECG_Filtered'].values

In [12]:
data.shape

(52770,)

In [13]:
print("Applying model ..")   
data = preprocess(data)
prob = model.predict(data)
ann = np.argmax(prob)
print("Record {} classified as {} with {:3.1f}% certainty".format(record,classes[ann],100*prob[0,ann]))

Applying model ..
Preprocessing recording ..
Record A00906 classified as ~ with 77.3% certainty


Well turns out my ECG readings classified as noisy